In [1]:
import itertools
import pickle
import numpy as np
from music21 import *
from matplotlib import pyplot as plt
from sklearn import svm, naive_bayes, neighbors
from sklearn.feature_extraction.text import CountVectorizer

import data_loader as loader


In [2]:
def fit(x_train,y_train, x_test, y_test,classifiers):
    for clf in classifiers:
        clf.fit(x_train,y_train)
        print(clf)
        print(f"acc: {clf.score(x_test,y_test)}","\n")

In [3]:
def find_k(x_train,y_train, x_test, y_test):
    best_acc = 0
    k = 0
    for i in range(1,30):
        k_nn = neighbors.KNeighborsClassifier(n_neighbors=i)
        k_nn.fit(x_train, y_train)
        acc = k_nn.score(x_test,y_test)
        if(acc>best_acc):
            best_acc = acc
            k = i
    print(f"best k = {k}, best acc = {best_acc}")
    return k

In [4]:
def fit_ngrams(x_train,y_train,x_test,y_test,n):    
    ngrams_train, ngrams_test = ngrams_vectorize(x_train, x_test,n)
    
    k = find_k(ngrams_train,y_train,ngrams_test,y_test)
    fit(ngrams_train,y_train,ngrams_test,y_test,
         [neighbors.KNeighborsClassifier(n_neighbors=k), naive_bayes.MultinomialNB(),svm.LinearSVC(dual=False)])
    

In [5]:
def ngrams_vectorize(x_train,x_test,n):
    print("n =",n)
    
    ngram_vectorizer = CountVectorizer(token_pattern="\d+", ngram_range=(1, n))
    ngrams_train = ngram_vectorizer.fit_transform(x_train)
    ngrams_test = ngram_vectorizer.transform(x_test)

    print("train shape =", ngrams_train.shape)
    print("test shape =", ngrams_test.shape)
    
    return ngrams_train, ngrams_test

In [6]:
def combine(x1,x2,x1_classes):
    result =[]
    for a,b in zip (x1,x2):
        result.append(x1_classes * b + a)
    return result

In [7]:
def combine_item(x1,x2,x1_classes):
    return x1_classes * x2 + x1

In [8]:
def unique_dict(in_list):
    dic = dict.fromkeys(in_list)
    for i,k in enumerate(dic.keys()):
        dic[k]=i
    return dic

In [9]:
def values_to_labels(in_list,dic):
    out_list = [dic[x] for x in in_list]
    return out_list        

In [10]:
def stringify_items(in_list):
    return [str(x) for x in in_list]

In [11]:
# Define known chords
chords_list = ([0], [0,4,7],[0,3,7],[0,5,7],[0,4,8],[0,3,6],[0,4,7,9],[0,3,7,9],
[0,4,7,10],[0,4,7,11],[0,3,7,10],[0,3,6,10],[0,3,6,9],[0,2,4,7,11],
[0,2,4,7,10],[0,1,4,7,10],[0,2,3,7,10])

# chord_types_list = [tone,major, minor, suspended, augmented, 
#                     diminished, major_sixth, minor_sixth, 
#                     dominant_seventh, major_seventh, 
#                     minor_seventh, half_diminished_seventh, 
#                     diminished_seventh, major_ninth, 
#                     dominant_ninth, dominant_minor_ninth, minor_ninth]

In [12]:
def find_chord(in_list):
    root = in_list[0]
    if(len(in_list)==1):
        return root
    costs = []
    for chord in chords_list:
        cost = 0
        for tone_p, tone_ch in itertools.zip_longest(in_list[1:],chord):
            if(tone_p == None or tone_ch == None):
                cost += 3
            else:
                cost+= abs(tone_p - tone_ch)
        costs.append(cost)
    i = list.index(costs,min(costs))
    return combine_item(root,i,12)
        

## Loading unprocessed data

In [13]:
%%capture
composerNames = ["debussy","tchaikovsky","mozart","victoria","beethoven"]
dataset_type = "chords_t"
x_train_ch, x_test_ch,y_train, y_test = loader.load(dataset_type,composerNames)

In [14]:
x_train = [str([chord[0] for chord in example ]) for example in x_train_ch]
x_test = [str([chord[0] for chord in example]) for example in x_test_ch]

In [18]:
x_train_d, x_test_d,y_train, y_test = loader.load("durations",composerNames)

In [19]:
# x_train = [str([find_chord(chord) for chord in example ]) for example in x_train_ch]
# x_test = [str([find_chord(chord) for chord in example])for example in x_test_ch]

In [20]:
# pitches_seq = list(itertools.chain.from_iterable(x_train_ch+x_test_ch))
# pitches_dict = unique_dict(pitches_seq)

# x_train_ch = [values_to_labels(example,pitches_dict) for example in x_train_ch]
# x_test_ch = [values_to_labels(example,pitches_dict) for example in x_test_ch]

# durations_seq = list(itertools.chain.from_iterable(x_train_d+x_test_d))
# durations_dict = unique_dict(durations_seq)

# x_train_d = [values_to_labels(example,durations_dict) for example in x_train_d]
# x_test_d = [values_to_labels(example,durations_dict) for example in x_test_d]

In [21]:
# x_train = list(map(str, combine(x_train_ch,x_train_d,len(pitches_dict))))
# x_test = list(map(str, combine(x_test_ch,x_test_d,len(pitches_dict))))

In [22]:
print(loader.get_data_counts(y_train,composerNames),loader.get_data_counts(y_test,composerNames))

{'debussy': 119, 'tchaikovsky': 165, 'mozart': 423, 'victoria': 188, 'beethoven': 380} {'debussy': 40, 'tchaikovsky': 54, 'mozart': 124, 'victoria': 61, 'beethoven': 61}


# Testing different models

In [23]:
x_train,y_train = loader.shuffle_data(x_train,y_train)
x_test,y_test = loader.shuffle_data(x_test,y_test)

for n in range(1,5):
    fit_ngrams(x_train,y_train,x_test,y_test,n)

n = 1
train shape = (1275, 12)
test shape = (340, 12)
best k = 6, best acc = 0.65
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=6, p=2,
           weights='uniform')
acc: 0.65 

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
acc: 0.6 

LinearSVC(C=1.0, class_weight=None, dual=False, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)
acc: 0.6529411764705882 

n = 2
train shape = (1275, 156)
test shape = (340, 156)
best k = 2, best acc = 0.7176470588235294
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=2, p=2,
           weights='uniform')
acc: 0.7176470588235294 

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
acc: 0.6588235294117647 

LinearSVC(C=1.0, class_weight=None, dual=False, fit_inter

# Training a model

In [16]:
ngram_vectorizer = CountVectorizer(token_pattern="\d+", ngram_range=(1, 4))
ngrams_train = ngram_vectorizer.fit_transform(x_train)
ngrams_test = ngram_vectorizer.transform(x_test)

print("train shape =", ngrams_train.shape)
print("test shape =", ngrams_test.shape)

clf = svm.LinearSVC(dual=False)
fit(ngrams_train, y_train,ngrams_test,y_test,[clf])


train shape = (1275, 22598)
test shape = (340, 22598)
LinearSVC(C=1.0, class_weight=None, dual=False, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)
acc: 0.7823529411764706 



# Saving a model

In [17]:
clfobj = loader.Classifier(clf,composerNames,ngram_vectorizer, "chords_t",loader.get_root)

In [18]:
with open("models/svm_4grams.dat","wb") as f:
    pickle.dump(clfobj,f)

In [ ]:

# x,files = loader.load_folder("midis","chords")

In [ ]:
# classes = list(set(y_test))
# test_data = list(zip(x_test,y_test))
# for label in classes:
#     label_data = [tupple[0] for tupple in test_data if tupple[1] == label][:10]
#     plt.plot(label_data)
#     plt.xlabel("Examples")
#     plt.show()